In [2]:
import numpy as np
from nanowire.optics.simulate import Simulator
from nanowire.optics.utils.utils import setup_sim
from nanowire.optics.utils.config import Config
import scipy.constants as consts

# Introduction

This is just a quick set of tests to figure out how to compute the absorption of an individual layer from the power fluxes at the top and bottom of each layer

In [3]:
conf = Config('AbsorptionTest.yml')
sim = Simulator(conf)
sim = setup_sim(sim)
Zo = consts.physical_constants['characteristic impedance of vacuum'][0]

DEBUG:nanowire.optics.simulate:Logger initialized
DEBUG:nanowire.optics.simulate:Building layer: Air
DEBUG:nanowire.optics.simulate:Layer Order 1
DEBUG:nanowire.optics.simulate:Building layer: ITO
DEBUG:nanowire.optics.simulate:Layer Order 2
DEBUG:nanowire.optics.simulate:Building layer: NW_AlShell
DEBUG:nanowire.optics.simulate:Layer Order 3
DEBUG:nanowire.optics.simulate:Building geometry in layer: NW_AlShell
DEBUG:nanowire.optics.simulate:Building object shell of type circle at order 1
DEBUG:nanowire.optics.simulate:Building object core of type circle at order 2
DEBUG:nanowire.optics.simulate:Building layer: NW_SiO2
DEBUG:nanowire.optics.simulate:Layer Order 4
DEBUG:nanowire.optics.simulate:Building geometry in layer: NW_SiO2
DEBUG:nanowire.optics.simulate:Building object shell of type circle at order 1
DEBUG:nanowire.optics.simulate:Building object core of type circle at order 2
DEBUG:nanowire.optics.simulate:Building layer: Substrate
DEBUG:nanowire.optics.simulate:Layer Order 5
DE

0


In [5]:
fluxes = sim.get_fluxes()

DEBUG:nanowire.optics.simulate:Computing fluxes ...
DEBUG:nanowire.optics.simulate:Computing fluxes through layer: Substrate
DEBUG:nanowire.optics.simulate:Computing fluxes through layer: NW_AlShell
DEBUG:nanowire.optics.simulate:Computing fluxes through layer: Air
DEBUG:nanowire.optics.simulate:Computing fluxes through layer: ITO
DEBUG:nanowire.optics.simulate:Computing fluxes through layer: NW_SiO2
DEBUG:nanowire.optics.simulate:Finished computing fluxes!


In [6]:
print(fluxes)

{'Substrate': ((0.5778386241618029+0j), 0j), 'NW_SiO2': ((0.6415681690545958-0.002683480964878013j), (-0.054474731321943956+0.002683480964878013j)), 'NW_AlShell': ((1.7313777694256565-0.0021892601347096793j), (-0.010730033470238865+0.0021892601347096793j)), 'NW_SiO2_bottom': ((0.632688624556471-0.0003835546834251788j), (-0.05485000039465573+0.0003835546834251788j)), 'Air': ((2+0j), (-0.15557938319276354+0j)), 'ITO': ((1.911743947569879+0.0008713251712626491j), (-0.06732333076256294-0.0008713251712626491j)), 'NW_AlShell_bottom': ((0.6199840123151336-0.0035553800448866954j), (-0.03289057458248516+0.0035553800448866954j)), 'ITO_bottom': ((1.7901404153159508-0.0023688997496048164j), (-0.06949267936052105+0.0023688997496048164j)), 'Substrate_bottom': ((0.08933583976596272+0j), 0j), 'Air_bottom': ((2+0j), (-0.15557938319276352+0j))}


In [7]:
out_ito = fluxes['ITO_bottom'][0] + fluxes['ITO_bottom'][1]
in_nw = fluxes['NW_AlShell'][0] + fluxes['NW_AlShell'][1]
print(out_ito)
print(in_nw)
print(fluxes['ITO_bottom'])
print(fluxes['NW_AlShell'])

(1.7206477359554297+0j)
(1.7206477359554175+0j)
((1.7901404153159508-0.0023688997496048164j), (-0.06949267936052105+0.0023688997496048164j))
((1.7313777694256565-0.0021892601347096793j), (-0.010730033470238865+0.0021892601347096793j))


In [8]:
ito_inc = np.absolute(fluxes['ITO'][0])
ito_ref = np.absolute(fluxes['Air_bottom'][1])
ito_ref = np.absolute(fluxes['ITO'][1])
ito_trans = np.absolute(fluxes['NW_AlShell'][0])
print(ito_inc)
print(ito_ref)
print(ito_trans)
absorbed = ito_inc - ito_ref - ito_trans
print(absorbed)

1.91174414613
0.0673289690439
1.73137915354
0.113036023548


In [9]:
total_incident_power = np.absolute(fluxes['Air'][0])
total_reflected_power = np.absolute(fluxes['Air'][1])
total_transmitted_power = np.absolute(sum(fluxes['Substrate_bottom']))
total_absorbed_power = total_incident_power - total_reflected_power - total_transmitted_power
print('Total Incident Power = {}'.format(total_incident_power))
print('Total Reflected Power = {}'.format(total_reflected_power))
print('Total Transmitted Power = {}'.format(total_transmitted_power))
print('Total Absorbed Power = {}'.format(total_absorbed_power))

Total Incident Power = 2.0
Total Reflected Power = 0.15557938319276354
Total Transmitted Power = 0.08933583976596272
Total Absorbed Power = 1.7550847770412736


In [11]:
summed_absorbed_power = 0
abs_dict_fluxmethod = {}
for layer, (forw_top, back_top) in fluxes.items():
    if '_bottom' in layer:
        continue
    bottom = layer+'_bottom'
    forw_bot, back_bot = fluxes[bottom] 
    print('-'*25)
    print('Layer: {}'.format(layer))
    print('Forward Top: {}'.format(forw_top))
    print('Backward Top: {}'.format(back_top))
    print('Forward Bottom: {}'.format(forw_bot))
    print('Backward Bottom: {}'.format(back_bot))
    P_in = forw_top + -1*back_bot
    P_out = forw_bot + -1*back_top
    #print('Power Entering Layer: {}'.format(P_in))
    #print('Power Leaving Layer: {}'.format(P_out))
    P_abs = P_in - P_out
    abs_dict_fluxmethod[layer] = .5*P_abs*(sim.period**2)/Zo
    #print('Absorbed in Layer: {}'.format(P_abs))
    summed_absorbed_power += P_abs
#print('Summed Absorption= {}'.format(np.absolute(summed_absorbed_power)))
print('-'*25)
print('Summed Absorption= {}'.format(summed_absorbed_power))
print('GaAs Absorption = {}'.format(abs_dict_fluxmethod['Substrate']))

-------------------------
Layer: Substrate
Forward Top: (0.5778386241618029+0j)
Backward Top: 0j
Forward Bottom: (0.08933583976596272+0j)
Backward Bottom: 0j
-------------------------
Layer: NW_SiO2
Forward Top: (0.6415681690545958-0.002683480964878013j)
Backward Top: (-0.054474731321943956+0.002683480964878013j)
Forward Bottom: (0.632688624556471-0.0003835546834251788j)
Backward Bottom: (-0.05485000039465573+0.0003835546834251788j)
-------------------------
Layer: NW_AlShell
Forward Top: (1.7313777694256565-0.0021892601347096793j)
Backward Top: (-0.010730033470238865+0.0021892601347096793j)
Forward Bottom: (0.6199840123151336-0.0035553800448866954j)
Backward Bottom: (-0.03289057458248516+0.0035553800448866954j)
-------------------------
Layer: Air
Forward Top: (2+0j)
Backward Top: (-0.15557938319276354+0j)
Forward Bottom: (2+0j)
Backward Bottom: (-0.15557938319276352+0j)
-------------------------
Layer: ITO
Forward Top: (1.911743947569879+0.0008713251712626491j)
Backward Top: (-0.0673

So if the loop above computed things properly, then summed absorbed power should be equal to the total absorbed power

In [12]:
diff = total_absorbed_power - summed_absorbed_power
print('Diff = {}'.format(diff))

Diff = (-5.861977570020827e-14+0j)


So they appear to be pretty close. This device didn't have an air layer below everything. I try adding an air layer below to
see if that improves the agreement between these two methods

In [11]:
below_conf = Config('AbsorptionTest_airbelow.yml')
below_sim = Simulator(below_conf)
below_sim = setup_sim(below_sim)

DEBUG:nanowire.optics.simulate:Logger initialized
DEBUG:nanowire.optics.simulate:Building layer: Air
DEBUG:nanowire.optics.simulate:Layer Order 1
DEBUG:nanowire.optics.simulate:Building layer: ITO
DEBUG:nanowire.optics.simulate:Layer Order 2
DEBUG:nanowire.optics.simulate:Building layer: NW_AlShell
DEBUG:nanowire.optics.simulate:Layer Order 3
DEBUG:nanowire.optics.simulate:Building geometry in layer: NW_AlShell
DEBUG:nanowire.optics.simulate:Building object shell of type circle at order 1
DEBUG:nanowire.optics.simulate:Building object core of type circle at order 2
DEBUG:nanowire.optics.simulate:Building layer: NW_SiO2
DEBUG:nanowire.optics.simulate:Layer Order 4
DEBUG:nanowire.optics.simulate:Building geometry in layer: NW_SiO2
DEBUG:nanowire.optics.simulate:Building object shell of type circle at order 1
DEBUG:nanowire.optics.simulate:Building object core of type circle at order 2
DEBUG:nanowire.optics.simulate:Building layer: Substrate
DEBUG:nanowire.optics.simulate:Layer Order 5
DE

In [12]:
fluxes = below_sim.get_fluxes()

DEBUG:nanowire.optics.simulate:Computing fluxes ...
DEBUG:nanowire.optics.simulate:Computing fluxes through layer: Substrate
DEBUG:nanowire.optics.simulate:Computing fluxes through layer: NW_SiO2
DEBUG:nanowire.optics.simulate:Computing fluxes through layer: ITO
DEBUG:nanowire.optics.simulate:Computing fluxes through layer: Air
DEBUG:nanowire.optics.simulate:Computing fluxes through layer: NW_AlShell
DEBUG:nanowire.optics.simulate:Computing fluxes through layer: Bottom_Air
DEBUG:nanowire.optics.simulate:Finished computing fluxes!


In [13]:
print(fluxes)

{'Substrate_bottom': ((43.60219291195291+0.6932449963104814j), (-15.07239896981021-0.6932449963104814j)), 'Bottom_Air_bottom': ((28.52979394214275+0j), 0j), 'NW_AlShell': ((814.5027719852595-0.797432675361371j), (-2.617026481935543+0.797432675361371j)), 'Bottom_Air': ((28.52979394214275+0j), 0j), 'NW_SiO2': ((299.1838150595673-1.7439443101523118j), (-14.872676558269914+1.7439443101523118j)), 'Substrate': ((282.21592462896365+0.5540592160286817j), (-2.7521874656798158-0.5540592160286817j)), 'NW_SiO2_bottom': ((294.8071086249769-0.620316201855837j), (-15.34337146169093+0.620316201855837j)), 'ITO': ((896.507349202051+0.5037645865747891j), (-26.29738076318455-0.5037645865747891j)), 'Air': ((930.4485521295919+0j), (-60.23858369069318+0j)), 'ITO_bottom': ((839.1481953357867-1.0767936010322146j), (-27.262449832452976+1.0767936010322146j)), 'Air_bottom': ((930.4485521295919+0j), (-60.23858369069321+0j)), 'NW_AlShell_bottom': ((291.9930683146283-1.3588527120678293j), (-7.681929813325565+1.35885

In [14]:
total_incident_power = np.absolute(fluxes['Air'][0])
total_reflected_power = np.absolute(fluxes['Air'][1])
total_transmitted_power = np.absolute(fluxes['Bottom_Air_bottom'][0])
total_absorbed_power = total_incident_power - total_reflected_power - total_transmitted_power
print('Total Incident Power = {}'.format(total_incident_power))
print('Total Reflected Power = {}'.format(total_reflected_power))
print('Total Transmitted Power = {}'.format(total_transmitted_power))
print('Total Absorbed Power = {}'.format(total_absorbed_power))

Total Incident Power = 930.4485521295919
Total Reflected Power = 60.23858369069318
Total Transmitted Power = 28.52979394214275
Total Absorbed Power = 841.680174496756


In [11]:
summed_absorbed_power = 0
abs_dict_fluxmethod = {}
for layer, (forw_top, back_top) in fluxes.items():
    if '_bottom' in layer:
        continue
    bottom = layer+'_bottom'
    forw_bot, back_bot = fluxes[bottom] 
    print('-'*25)
    print('Layer: {}'.format(layer))
    print('Forward Top: {}'.format(forw_top))
    print('Backward Top: {}'.format(back_top))
    print('Forward Bottom: {}'.format(forw_bot))
    print('Backward Bottom: {}'.format(back_bot))
    P_in = forw_top + -1*back_bot
    P_out = forw_bot + -1*back_top
    #print('Power Entering Layer: {}'.format(P_in))
    #print('Power Leaving Layer: {}'.format(P_out))
    P_abs = P_in - P_out
    abs_dict_fluxmethod[layer] = .5*P_abs*(sim.period**2)/Zo
    #print('Absorbed in Layer: {}'.format(P_abs))
    summed_absorbed_power += P_abs
#print('Summed Absorption= {}'.format(np.absolute(summed_absorbed_power)))
print('-'*25)
print('Summed Absorption= {}'.format(summed_absorbed_power))
print('GaAs Absorption = {}'.format(abs_dict_fluxmethod['Substrate']))

-------------------------
Layer: Substrate
Forward Top: (0.5778386241618029+0j)
Backward Top: 0j
Forward Bottom: (0.08933583976596272+0j)
Backward Bottom: 0j
-------------------------
Layer: NW_SiO2
Forward Top: (0.6415681690545958-0.002683480964878013j)
Backward Top: (-0.054474731321943956+0.002683480964878013j)
Forward Bottom: (0.632688624556471-0.0003835546834251788j)
Backward Bottom: (-0.05485000039465573+0.0003835546834251788j)
-------------------------
Layer: NW_AlShell
Forward Top: (1.7313777694256565-0.0021892601347096793j)
Backward Top: (-0.010730033470238865+0.0021892601347096793j)
Forward Bottom: (0.6199840123151336-0.0035553800448866954j)
Backward Bottom: (-0.03289057458248516+0.0035553800448866954j)
-------------------------
Layer: Air
Forward Top: (2+0j)
Backward Top: (-0.15557938319276354+0j)
Forward Bottom: (2+0j)
Backward Bottom: (-0.15557938319276352+0j)
-------------------------
Layer: ITO
Forward Top: (1.911743947569879+0.0008713251712626491j)
Backward Top: (-0.0673

In [16]:
diff = total_absorbed_power - summed_absorbed_power
print('Diff = {}'.format(diff))

Diff = 4.945377440890297e-11
